In [1]:
library(RNifti)
library(dplyr)
library(oro.nifti)


Attaching package: ‘dplyr’


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union


oro.nifti 0.11.4


Attaching package: ‘oro.nifti’


The following object is masked from ‘package:dplyr’:

    slice


The following object is masked from ‘package:RNifti’:

    origin




In [2]:
results_dir <- "scratch/tractoflow_hcp_dwi/results/"
patients <- list.files(results_dir, full.names = TRUE)
patients <- patients[grep("^\\d{6}$", basename(patients))]
patient_data <- read.csv("scratch/tractoflow_hcp_dwi/patient_data.csv")
patient_data_gender <- read.csv("scratch/tractoflow_hcp_dwi/patient_data_gender.csv")
subj_data <- patient_data[c("Subject", "Age_in_Yrs", "BMI")]
subj_data_gender <- patient_data_gender[c("Subject", "Gender")]
subj_data=merge(subj_data, subj_data_gender)
rois <- c("fa_anterior_hypothalamic_area", "fa_arcuate_hypothalamic_nucleus", "fa_dorsal_periventricular_hypothalamic_nucleus", "fa_dorsomedial_hypothalamic_nucleus", "fa_lateral_hypothalamus", "fa_mammillary_bodies", "fa_medial_preoptic_nucleus", "fa_paraventricular_nucleus", "fa_periventricular_hypothalamic_nucleus", "fa_posterior_hypothalamic_nucleus", "fa_suprachiasmatic_hypothalamic_nucleus", "fa_supraoptic_hypothalamic_nucleus", "fa_tuberomammillary_hypothalamic_nucleus", "fa_ventromedial_hypothalamus")

In [3]:
roi_data <- data.frame(Subject=character(0))
for (i in 1:length(patients)) {
    var_names_lh <-list()
    var_names_rh <-list()
    data_lh <-list()
    data_rh <-list()
    avg_vals <- list()
    for (roi in 1:length(rois)) {
        if (i==1) {
            roi_data[[rois[roi]]] <- numeric(0)
        }
        var_names_lh[[roi]] <- file.path("scratch", "tractoflow_hcp_dwi", "subject_rois_bin", basename(patients[i]), "fa_rois", paste0(basename(patients[i]), "_", rois[roi], "_lh_resampled.nii.gz"))
        var_names_rh[[roi]] <- file.path("scratch", "tractoflow_hcp_dwi", "subject_rois_bin", basename(patients[i]), "fa_rois", paste0(basename(patients[i]), "_", rois[roi], "_rh_resampled.nii.gz"))
        data_lh[[roi]] <- readNifti(var_names_lh[[roi]])
        data_rh[[roi]] <- readNifti(var_names_rh[[roi]])
        non_zero_vals_lh <- data_lh[[roi]]
        non_zero_vals_rh <- data_rh[[roi]]
        non_zero_vals_lh <- non_zero_vals_lh[non_zero_vals_lh > 0]
        non_zero_vals_rh <- non_zero_vals_rh[non_zero_vals_rh > 0]
        avg_val <- (mean(non_zero_vals_lh)+mean(non_zero_vals_rh))/2
        avg_vals <- append(avg_vals, avg_val)
        temp_data <- data.frame(Subject=basename(patients[i]), avg_vals)
    }
    colnames(temp_data)=c("Subject", rois)
    roi_data <- rbind(roi_data, temp_data)
}
subj_data <- merge(subj_data, roi_data)
write.csv(subj_data, "scratch/tractoflow_hcp_dwi/linear_regression/HCP_fa_data_bin.csv")